<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LSH" data-toc-modified-id="LSH-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>LSH</a></span><ul class="toc-item"><li><span><a href="#Part-I:-computing-similarity-amongst-text-documents" data-toc-modified-id="Part-I:-computing-similarity-amongst-text-documents-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Part I: computing similarity amongst text documents</a></span></li><li><span><a href="#Part-II:-computing-similarity-amongst-vectors" data-toc-modified-id="Part-II:-computing-similarity-amongst-vectors-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Part II: computing similarity amongst vectors</a></span></li></ul></li><li><span><a href="#Part-I:-computing-similarity-amongst-text-documents" data-toc-modified-id="Part-I:-computing-similarity-amongst-text-documents-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part I: computing similarity amongst text documents</a></span><ul class="toc-item"><li><span><a href="#Shingling" data-toc-modified-id="Shingling-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Shingling</a></span></li><li><span><a href="#Adding-Locality-Sensitive-Hashing:-preliminary-band-structure-theory,-how-to-choose-band-size" data-toc-modified-id="Adding-Locality-Sensitive-Hashing:-preliminary-band-structure-theory,-how-to-choose-band-size-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Adding Locality Sensitive Hashing: preliminary band-structure theory, how to choose band size</a></span></li><li><span><a href="#Effects-of-changing-$b,-r$-at-fix-$n$" data-toc-modified-id="Effects-of-changing-$b,-r$-at-fix-$n$-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Effects of changing $b, r$ at fix $n$</a></span></li></ul></li><li><span><a href="#Effects-of-changing-$n$-while-adjusting-$b,-r$-to-keep-$P-=-\frac{1}{2}$-(we-can-solve-the-optimal-values-$r,-b$)" data-toc-modified-id="Effects-of-changing-$n$-while-adjusting-$b,-r$-to-keep-$P-=-\frac{1}{2}$-(we-can-solve-the-optimal-values-$r,-b$)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Effects of changing $n$ while adjusting $b, r$ to keep $P = \frac{1}{2}$ (we can solve the optimal values $r, b$)</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Brute-force-banded-candidate-pair-function,-for-checking-hash-method-later" data-toc-modified-id="Brute-force-banded-candidate-pair-function,-for-checking-hash-method-later-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Brute force banded candidate pair function, for checking hash method later</a></span></li></ul></li></ul></li><li><span><a href="#cosine-similarity" data-toc-modified-id="cosine-similarity-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>cosine similarity</a></span><ul class="toc-item"><li><span><a href="#Generate-data" data-toc-modified-id="Generate-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Generate data</a></span></li><li><span><a href="#LSH" data-toc-modified-id="LSH-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>LSH</a></span><ul class="toc-item"><li><span><a href="#Find-r,-b" data-toc-modified-id="Find-r,-b-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Find r, b</a></span></li><li><span><a href="#Compute-LSH-pairs" data-toc-modified-id="Compute-LSH-pairs-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Compute LSH pairs</a></span></li></ul></li></ul></li><li><span><a href="#Refactor-Cosine-Similarity" data-toc-modified-id="Refactor-Cosine-Similarity-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Refactor Cosine Similarity</a></span></li></ul></div>

# LSH

[ref](http://192.168.14.14:8000/user/joetsai/notebooks/work/yulong/cs246_mining_massive_datasets/lsh_jaccard/LocalitySensitiveHashing.ipynb)

python implementation of LSH

dataset

4 articles pulled from CNN

concatenation of 3 of those, this one artificialy produce some high similarity scores

outline

## Part I: computing similarity amongst text documents
1. Introduce a shingle function. Clean and split each text file into a set of K-shingles
2. Compute the exact Jaccard similarity (intersection over union) between all pairs
3. Create and apply a MinHashing class:
    1. Initialize with a dictionary of key-value pairs for the shingles
    2. Apply "universal hashing" to perform minhashing on a shingle set
    3. can be called like a function to compute a **signature matrix**
4. Evaluate MinHashing effectiveness by computing scores of all pairs
5. Introduce LSH for finding **candidate pairs**, i.e. use a banded signature matrix to find all pairs whose similarity is likely above a threshold
6. Make this efficient, by using hash table for band, column ids, allowing O(n) comparison

## Part II: computing similarity amongst vectors

* Afterwards, we provide an additional LSH family for Euclidean spaces, namely cosine similarity. 
* This is used to ascertain the similarity of vectors in a D-dimensional space.
* Can be implemented using the *Random Hyperplanes* hashing method.

In [1]:

%config Completer.use_jedi = False

import os
import time
import itertools
import collections
import numpy as np
from os.path import join as PJ


%matplotlib inline

import matplotlib.pyplot as plt

plt.style.use('ggplot')

from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import cufflinks as cf
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import pandas as pd
import numpy as np
from nptyping import NDArray
import plotly.express as px
import cufflinks
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

# Part I: computing similarity amongst text documents

## Shingling

1. Create k shingles of for each document
2. Calculate total shingles,
3. Conaider Input Matrix (Shingles x documents) -  $I_{S \times D}$
4. Create MinHasher(the minsh hash object)
6. Hash the document into Signature Matrix - $M_{\alpha \times D}$

where $\alpha$ is the `n_signature`, you should make $\alpha << S$

Because we do min-hashing do preserve the similarity of Jaccard Similarity
and do the dimension reduction 

In [2]:

HOME = os.getcwd()
TARGET = os.path.join(HOME,'data', 'sampledocs_jaccard')

print(HOME,TARGET, sep='\n')

documents = []
for article in os.listdir(TARGET):
    if article == 'stopwords':
        continue
    path = os.path.join(TARGET, article)
    with open(path, 'r') as file:
        documents.append(file.read())
        
stopwords = []
with open(os.path.join(TARGET, 'stopwords'), 'r') as file:
    for line in file:
        stopwords.append(line.strip())
        
for i, doc in enumerate(documents):
    doc = doc.strip().replace('\n', ' ').lower()
    for word in stopwords:
        doc = doc.replace(' '+word+' ', ' ')
    documents[i] = doc

print(f"Average char-length: \
{np.mean(np.array([len(x) for x in documents]))}")
print(f"Min char-length: {min(len(x) for x in documents)}")
print(f"Max char-length: {max(len(x) for x in documents)}")

/home/joetsai/work/yulong/cs246_mining_massive_datasets/implementation/lsh
/home/joetsai/work/yulong/cs246_mining_massive_datasets/implementation/lsh/data/sampledocs_jaccard
Average char-length: 3651.6
Min char-length: 2412
Max char-length: 5873


In [3]:
# create K-shingles by sliding window approach
# TODO [joe] - make this sparse matrix
def getShingles(str1, K=5):
    d1 = set()
    for i in range(len(str1)-K):
        d1.add(str1[i:i+K])
    print(f"Found {len(d1)} unique shingles, out of {len(str1)} possible.")
    return d1
doc_shingles = [getShingles(s, 5) for s in documents]

Found 2782 unique shingles, out of 3673 possible.
Found 2091 unique shingles, out of 3291 possible.
Found 1918 unique shingles, out of 2412 possible.
Found 3953 unique shingles, out of 5873 possible.
Found 2060 unique shingles, out of 3009 possible.


In [4]:
def jaccardSim(d1,d2):
    return len(d1.intersection(d2))/len(d1.union(d2))

# itertools.combinations finds all (,n) n-pairs
# then we use a map op on the tuples with jaccardSim
pairs = itertools.combinations(documents, 2)
pair_labels = []
pair_sims = []
for x1, x2 in itertools.combinations(zip(range(len(doc_shingles)),doc_shingles), 2):
    pair_labels.append((x1[0],x2[0]))
    pair_sims.append(jaccardSim(x1[1],x2[1]))
    
print(f"**~~~~~~ True similarity scores ~~~~~~**")
print("Pair\tScore")
print("-"*14)
for pair, score in zip(pair_labels, pair_sims):
    print(f"{pair}\t{score:.3f}")

**~~~~~~ True similarity scores ~~~~~~**
Pair	Score
--------------
(0, 1)	0.081
(0, 2)	0.052
(0, 3)	0.083
(0, 4)	0.069
(1, 2)	0.051
(1, 3)	0.294
(1, 4)	0.093
(2, 3)	0.400
(2, 4)	0.050
(3, 4)	0.336


In [5]:
# Take union of all sets. Convert to an array and assign
# each element an integer based on position in array
fullset = set.union(*doc_shingles)
shingle_dict = dict(zip(list(fullset),range(len(fullset))))
print(f"There are {len(shingle_dict)} shingles")

There are 7534 shingles


In [6]:
doc_shingles[0]

{'rked ',
 'list"',
 'dence',
 'edit,',
 ', leg',
 'nt ho',
 'ntimi',
 'ging ',
 'n -- ',
 'o, go',
 'n res',
 "t's c",
 'ss co',
 'd con',
 's mee',
 'ge de',
 'its t',
 ' prep',
 'ice a',
 'y kno',
 'inspi',
 'iden ',
 'ee co',
 'enate',
 'ns de',
 'see c',
 'me co',
 ' cost',
 'ys tr',
 '0 hou',
 's off',
 'ng ho',
 'etrib',
 'vernm',
 'ent, ',
 'ess b',
 'urgin',
 'tion.',
 'ce. r',
 ', fea',
 'ument',
 "p's b",
 'ost. ',
 's pol',
 ' cryi',
 'le ar',
 'dent ',
 'creta',
 'ate e',
 'alf. ',
 'ebase',
 'nal l',
 ' fiat',
 'preme',
 'ed bo',
 'ge fi',
 ' too,',
 'nce c',
 'how t',
 'cans ',
 'mbers',
 'p. mi',
 'unnin',
 'ibuti',
 'ld tr',
 "ump's",
 'ws ri',
 'terou',
 'aw oa',
 ' unco',
 'ed tr',
 'es an',
 'uits ',
 "iden'",
 'onser',
 '-hour',
 ' work',
 'iani ',
 'nt, b',
 'ic am',
 'ds ev',
 'y. ye',
 'y, du',
 'ryday',
 'ing s',
 ' righ',
 'e cha',
 'nt.al',
 'ed so',
 'ls ac',
 'selve',
 'ters ',
 'sly a',
 'mly u',
 'nce, ',
 've su',
 'gratu',
 ' -- m',
 'ts, p',
 ' fina',


In [7]:
# shingle = 5
# shingle_dict
# len(doc_shingles[0])
############ Examples #############
# https://numpy.org/doc/1.20/reference/generated/numpy.dot.html?highlight=dot#numpy.dot

# a @ b means a, b are two dimentional matrix,
# and the result is the same as np.dot(a,b)

a = np.array([[1, 0], [0, 1]])
b = np.array([[4, 1], [2, 2]])


print(
    'shingle of doc 1 example : ',
    list(doc_shingles[0])[:5],
    np.dot(a,b),
    a @ b,
    sep='\n'
)

shingle of doc 1 example : 
['rked ', 'list"', 'dence', 'edit,', ', leg']
[[4 1]
 [2 2]]
[[4 1]
 [2 2]]


In [8]:
# Create a hash function
# define as a callable class, so that we only initialize 
# random function once
# This is only Minhasher



n_sig = lambda x: x >= 0
n_doc = lambda x: x >= 0



class HashManager():
    def __init__(self, shingle_dict : dict, n_signature : int) -> None:
        self.shingle_dict = shingle_dict
        self.N = len(shingle_dict)
        self.n_signature = n_signature
        # create random integer from zero to N
        # TODO
        self.params = np.random.randint(self.N,
                                        size=[n_signature, 2])
    
    def _permuteRow(self, row):
        # TODO
        return self.params @ np.array([1, row]) % self.N
    
    def __call__(self, docs) -> np.ndarray:
        """
        building signature matrix by min hash
        return np.array shape : (self.n_signature, self.N)
        """
        # initialize signature matrix
        sig = np.full(
            (self.n_signature, len(docs)),
             np.inf
            )
        
        # each doc in docs is assumed to be an iterable object
        # we hash all the docs into signature matrix
        for doc_i, doc in enumerate(docs):
            for shingle in doc:
#                 print(list(doc)[:5], shingle)
                orig_row = shingle_dict[shingle]
                curr_col = self._permuteRow(orig_row)
                sig[:, doc_i] = np.minimum(
                    sig[:, doc_i],
                    curr_col
                )
        return sig.astype(int)
    
# run some tests

try:
    n_signature = 100
    print("Initilization")
    min_hasher = HashManager(shingle_dict,
                             n_signature=n_signature)
    print("passed")
    
    print('parametets is the right shape')
    assert(min_hasher.params.shape == (n_signature, 2))
    print('passed')
    
    print("permuting a row integer returns array")
    curr_col = min_hasher._permuteRow(3)
    assert(curr_col.shape == (n_signature, ))
    print('passed')
    
    print("Compute minhashed signature matrix : ")
    res = min_hasher(doc_shingles)
    assert res.shape == (n_signature, len(doc_shingles))
    print("passed")
    
except Exception as e:
    print(e.args)

Initilization
passed
parametets is the right shape
passed
permuting a row integer returns array
passed
Compute minhashed signature matrix : 
passed


In [9]:
def trueSimScores(doc_shingles):
    pair_labels = []
    pair_sims = []
    idxs = range(len(doc_shingles))
    for x1, x2 in itertools.combinations(zip(idxs,doc_shingles), 2):
        pair_labels.append((x1[0], x2[0]))
        pair_sims.append(jaccardSim(x1[1], x2[1]))
    return dict(zip(pair_labels, pair_sims))
    
def sigSimScores(sig_mat):
#     cols = [sig_mat[:,i] for i in range(sig_mat.shape[1])]
    cols = sig_mat.T
    idxs = range(sig_mat.shape[1])
    
    pair_labels = []
    pair_sims = []
    for (i,col1), (j,col2) in itertools.combinations(zip(idxs, cols),2):
        pair_labels.append((i,j))
        pair_sims.append(np.mean(col1==col2))
    
    return dict(zip(pair_labels, pair_sims))

def printScoreComparison(true_dict, approx_dict):
    print(f"**~~~~~~ Similarity score comparison ~~~~~~**")
    print("Pair\t\tApprox\t\tTrue\t\t%Error")
    for pair, true_value in true_dict.items():
        approx_value = approx_dict[pair]
        err = 100*abs(true_value-approx_value)/true_value
        print(f"{pair}\t\t{approx_value:.3f}\t\t{true_value:.3f}\t\t{err:.2f}")

def candidatePairs(score_dict, threshold):
    return set(pair for pair, scr in score_dict.items() if scr>=threshold)

def accMatrix(true_dict, approx_dict, threshold):
    true_pairs = candidatePairs(true_dict, threshold)
    approx_pairs = candidatePairs(approx_dict, threshold)
    false_negatives = len(true_pairs - approx_pairs)
    false_positives = len(approx_pairs - true_pairs)
    print(f"False negatives: {false_negatives}")
    print(f"False negtive sample : ", true_pairs - approx_pairs)
    print(f"Potential false positives: {false_positives}")
    print(f"Potential flase positive sample : ", approx_pairs - true_pairs)


In [10]:
min_hasher = HashManager(shingle_dict,
                             n_signature=20)

sig_mat = min_hasher(doc_shingles)
print(sig_mat.shape)
true_score_dict = trueSimScores(doc_shingles)
approx_score_dict = sigSimScores(sig_mat)
printScoreComparison(true_score_dict, approx_score_dict)

print("True pairs:",candidatePairs(true_score_dict, 0.25))
print("Candidate pairs:",candidatePairs(approx_score_dict, 0.25))
accMatrix(true_score_dict, approx_score_dict, 0.25)



(20, 5)
**~~~~~~ Similarity score comparison ~~~~~~**
Pair		Approx		True		%Error
(0, 1)		0.250		0.081		208.77
(0, 2)		0.150		0.052		190.19
(0, 3)		0.150		0.083		80.78
(0, 4)		0.050		0.069		27.40
(1, 2)		0.200		0.051		289.08
(1, 3)		0.500		0.294		69.78
(1, 4)		0.200		0.093		114.52
(2, 3)		0.550		0.400		37.43
(2, 4)		0.150		0.050		200.71
(3, 4)		0.400		0.336		18.97
True pairs: {(3, 4), (1, 3), (2, 3)}
Candidate pairs: {(0, 1), (1, 3), (2, 3), (3, 4)}
False negatives: 0
False negtive sample :  set()
Potential false positives: 1
Potential flase positive sample :  {(0, 1)}


A case about false positive and false negtvie

True pairs: {(3, 4), (1, 3), (2, 3)}

Candidate pairs: {(0, 3), (1, 3), (2, 3), (0, 2)}

False negatives: 1

False negtive sample :  {(3, 4)}

Potential false positives: 2

Potential flase positive sample :  {(0, 3), (0, 2)}


## Adding Locality Sensitive Hashing: preliminary band-structure theory, how to choose band size

## Effects of changing $b, r$ at fix $n$

We use a band structre on the signature matrix, If the sigmature matrix $M_{\alpha \times D}$ have $\alpha$ rows, then we devide it into $b$ bands each width $r$, such that

$$
\alpha = br
$$

Let $p$ be the **true similarity ** between a pair

The probability (whole band dosn't be hashed into a bucket)

$$
\text{prob[one band doesn't match]} = 1 - p^{r}
$$

e.g. if $p=0.8, r=5$, the prob [one band dosn't match] = $1-{0.8}^{5} = 0.67$ 

the probability that NOE of $b$ bands match 

$$
\text{prob[no bands match]} = (1 - p^{r})^{b}
$$

e.g. if $p=0.8, r=5, n=20, b=4$, the prob [no bands match] = $ (1-{0.8}^{5})^{4} = (0.67)^{4} = 0.20$ which is small chance when we make $b$ bigger

The prob that **at least one band matches**

$$
\text{prob at least 1 band match} = 1 - (1-p^{r})^{b}
$$


e.g. if $p=0.8, r=5, n=20, b=4$, the prob [at least 1 band match] = $ 1 - 0.20 = 0.80$ 

All right, Why we caculate those?

from some math

**Hash to the same bucket for >= 1 band** - candidate pair, it's actually the prob $1 - (1-p^{r})^{b}$

**Physical meaning**

1. Hash a whole band, the pairs in the same bucket 

    --> reduce false positive(increase precision),
    
    when $r$ increase, false positive decrease

2. at lease 1 band match 
    
    --> reduce false negtive (increase recall / coverage of the pairs)
    
    when $b$ increase, false negtive decrease

In [11]:
from typing import Union
def prob_at_least_match_one_bucket(
    x : Union[float, np.ndarray], 
    b : int,
    r : int
) -> Union[float, np.ndarray]:
    """
    calculate probability of at lease match 1 bands
    whcih gives candidate pair in LSH algorithm
    
    x(float) : similarity threshold, x in [0, 1]
    b(int) : number of bands
    r(int) : rows per band
    you should notice that 
    the number of signatures(a.k.a rows of signature mtrix) = b * r
    
    Use formula in production, use varaible comment for expereiment
    formula : 1 - (1-x **r) **b
    """
    one_band_dosnt_match =  (1 - x ** r)
    no_bands_match = one_band_dosnt_match ** b
    at_least_match_one_band = 1 - no_bands_match
    return at_least_match_one_band

In [12]:
cufflinks.themes.THEMES.keys(),cf.get_scales().keys()

(dict_keys(['ggplot', 'pearl', 'solar', 'space', 'white', 'polar', 'henanigans']),
 dict_keys(['brbg', 'prgn', 'piyg', 'puor', 'rdbu', 'rdgy', 'rdylbu', 'rdylgn', 'spectral', 'paired', 'set3', 'dflt', 'original', 'plotly', 'henanigans', 'accent', 'dark2', 'pastel1', 'pastel2', 'set1', 'set2', 'blues', 'bugn', 'bupu', 'gnbu', 'greens', 'greys', 'orrd', 'oranges', 'pubu', 'pubugn', 'purd', 'purples', 'rdpu', 'reds', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd', 'ggplot', 'polar']))

In [13]:
# x stands the ture similarity between two sets
x = np.linspace(0, 1, 100)

# suppose we have signature matrix 200 rows(we use 200 hash tables)
n=200



benchmark = {
    'similarity_threshold':x
}

for b, r in [
    (1,1),
    (1, 200),
    (2, 100),
    (4, 50),
    (8, 25),
    (10, 20),
    (20, 10),
    (25, 8),
    (50, 4),
    (100, 2),
    (200, 1),
    ]:
    benchmark[f'b_{b}_r_{r}'] = prob_at_least_match_one_bucket(x=x, b=b, r=r)



fig = (
    pd.DataFrame(benchmark)
    .set_index('similarity_threshold')
    .iplot(
        # prob_at_lease_match_one_band p
        xTitle = r'similarity between two sets ',
        # prob_at_lease_match_one_band 
        yTitle = r'prob_at_lease_match_one_band',
        title = r'$b \text{ versus } r\text{ at fix } n$',
        asFigure = True,
        colorscale='plotly'
          )
#     https://plotly.com/python/shapes/
    
    .add_trace(go.Scatter(
                x=[0.5,0.5], y=[1.1,1.1],
                text=[r"$\text{similarity threshold } (p)$"],
                mode="text",
                name='similarity threshold'
    ))
    .add_shape(
        type="line",x0=0.5, x1=0.5, y0=0, y1=1,
        line=dict(color='DarkOrange', width=3),
        name='similarity threshold'
              )
)



fig.show()

 The verticalline stands **true similarity threshold**,
 
 $b=1, r=1$ stands the min hash func(linear relation to similarity)
 
 area under curve $P(x < p)$ : **potential false postive** 
 
 area under curve $P(x > p)$ : **potential false negtive** - means the positive should be attendcapture by algo, but predicted negtive
  
 The goal of tuning $b, r$ is find a curve minimize you care about

# Effects of changing $n$ while adjusting $b, r$ to keep $P = \frac{1}{2}$ (we can solve the optimal values $r, b$)

Suppose we fix $n, b, r$, and we can determine at whcih approximate point $p$ should be

if we pick a cadidate pair probability $P$, then we can decide $p$

e.g. $P=\frac{1}{2}$

$$ /frac{1}{2} = 1 - (1 - p^{r})^{b} $$

$$ 1 - p^{r} = 2^{\frac{-1}{b}} $$

$$ p = (1 - 2^{\frac{-1}{b}})^{\frac{1}{r}} $$

Using $2 = e^{ln2}$ tech

$$p = (1 - e^{\frac{-1}{b} ln2}) ^{\frac{1}{r}} \approx (\frac{1}{b})^{\frac{1}{r}} \cdot \text{const}$$

Finally, if we fix $r$ and $p$ , we can find the require bands to be about 

$$b \approx \frac{1}{p^{r}}$$

increase $r$

* move the curve right
* more false negtive (model is more focus on precision) - 嚴謹
* means lower chance to match

increase $b$

* move the curve lieft
* more false positive(model is more focus on coverage) - 寬鬆
* more higher change to match

increase $n$

* curve approaches a step function
* fewer false anythings
* alwaya good

you can notice the optimal function in [Datasketch](https://github.com/ekzhu/datasketch/blob/master/datasketch/lsh.py)

which use a brute force approach to compute the optimal values(but it's ok)


```python
def _optimal_param(threshold, num_perm, false_positive_weight,
        false_negative_weight):
    '''
    Compute the optimal `MinHashLSH` parameter that minimizes the weighted sum
    of probabilities of false positive and false negative.
    '''
    min_error = float("inf")
    opt = (0, 0)
    for b in range(1, num_perm+1):
        max_r = int(num_perm / b)
        for r in range(1, max_r+1):
            fp = _false_positive_probability(threshold, b, r)
            fn = _false_negative_probability(threshold, b, r)
            error = fp*false_positive_weight + fn*false_negative_weight
            if error < min_error:
                min_error = error
                opt = (b, r)
    return opt
```

In [14]:
# x stands the ture similarity between two sets
x = np.linspace(0, 1, 100)

# suppose we have signature matrix 200 rows(we use 200 hash tables)
n=200



benchmark = {
    'similarity_threshold':x
}

for b, r in [
    # note the if you wanna fix the corss over point the same
    # you gonna keep the relation b ~ \frac{1}{p^{r}}
    (5, 3),
    (40, 6),
    (150, 8),
    (700, 10),
    (1500, 11),
    (50000, 16),
    (2000000,21)
    ]:
    n = b * r
    benchmark[f'b_{b}_r_{r}_n_{n}'] = prob_at_least_match_one_bucket(x=x, b=b, r=r)



fig = (
    pd.DataFrame(benchmark)
    .set_index('similarity_threshold')
    .iplot(
        # prob_at_lease_match_one_band p
        xTitle = r'similarity between two sets ',
        # prob_at_lease_match_one_band 
        yTitle = r'prob_at_lease_match_one_band',
        title = r'$\text{ Increasing } n \text{ by fix } b \approx \frac{1}{p^{r}}$',
        asFigure = True,
        colorscale='plotly'
          )
#     https://plotly.com/python/shapes/
    
    .add_trace(go.Scatter(
                x=[0.5,0.5], y=[1.1,1.1],
                text=[r"$\text{similarity threshold } (p)$"],
                mode="text",
                name='similarity threshold'
    )
    )
    .add_shape(
        type="line",x0=0.5, x1=0.5, y0=0, y1=1,
        line=dict(color='DarkOrange', width=3),
        name='similarity threshold'
              )
    .add_shape(
        type="line",x0=0.0, x1=1, y0=0.5, y1=0.5,
        line=dict(color='DarkOrange', width=3),
        name='similarity threshold'
              )
)



fig.show()

### Brute force banded candidate pair function, for checking hash method later

In [15]:
# TODO

# do some review

In [16]:
from typing import Set, Tuple

def bandedCandidatePair(sig_1, sig_2, b,r):
    """
    a helper function for bandedCandidatePairs
    Return a boolean if two columns are a candidate pair
    """
    n_sig = len(sig_1)
    assert (n==b*r)
    assert (n==len(sig_2))
    truth_array = (sig_1 == sig_2)
    return any(
        all(band)
        for band
        in np.array_split(truth_array, b)
              )

def bandedCandidatePairs(sig_mat : np.ndarray,
                         b : int,
                         r : int) -> Set[tuple]:
    """
    sig_mat : shape n_signature, n_docs
    
    return {(0, 1), (0, 2)} 
    means 0, 1 are candidate pairs, 0, 2 are candidate pairs
    """
    pairs = set()
    
    n_doc = sig_mat.shape[1]
    doc_idx = range(n_doc)
    signatures = [sig_mat[:, doc_idx] for doc_idx in range(n_doc)]
    for (i, sig_1), (j, sig_2) in itertools.combinations(
        zip(doc_idx, signatures), 2):
        if bandedCandidatePair(sig_1, sig_2, b, r):
            pairs.add((i, j))
    return pairs


# set p = 0.3 arbitrarily
p = 0.3
n = 120
b = 30
r = 4


min_hasher = HashManager(shingle_dict,
                             n_signature=n)

sig_mat = min_hasher(doc_shingles)
print(sig_mat.shape)

# see how many candidate pairs we got right!
true_score_dict = trueSimScores(doc_shingles)
approx_score_dict = sigSimScores(sig_mat)
print("True pairs:",candidatePairs(true_score_dict, p))
print("LSH pairs:",bandedCandidatePairs(sig_mat, b, r))
print("Vanilla MinHash pairs:",candidatePairs(approx_score_dict, p))
# accMatrix(true_score_dict, approx_score_dict, 0.4)

# sig_mat = hm(doc_shingles, n)
# true_score_dict = trueSimScores(doc_shingles)
printScoreComparison(true_score_dict, approx_score_dict)

(120, 5)
True pairs: {(3, 4), (2, 3)}
LSH pairs: {(1, 3)}
Vanilla MinHash pairs: {(3, 4), (1, 3), (2, 3)}
**~~~~~~ Similarity score comparison ~~~~~~**
Pair		Approx		True		%Error
(0, 1)		0.150		0.081		85.26
(0, 2)		0.108		0.052		109.59
(0, 3)		0.167		0.083		100.87
(0, 4)		0.117		0.069		69.39
(1, 2)		0.125		0.051		143.18
(1, 3)		0.408		0.294		38.66
(1, 4)		0.167		0.093		78.77
(2, 3)		0.358		0.400		10.46
(2, 4)		0.083		0.050		67.06
(3, 4)		0.417		0.336		23.93


In [17]:
# Let's perform a fast version

def fastCandidatePairs(sig_mat : np.ndarray,
                       b : int, r : int) -> Set[tuple]:
    """
    sig_mat : shape n_signature, n_docs
    
    return {(0, 1), (0, 2)} 
    means 0, 1 are candidate pairs, 0, 2 are candidate pairs
    """
    
    n_sig, n_doc = sig_mat.shape
    
    assert(n == b * r)
    
    hashbuckets = collections.defaultdict(set)
    
    # splits signatures by band
    bands = np.array_split(sig_mat, b, axis=0) 
    
    for i, band in enumerate(bands):
        for j in range(n_doc):
            band_id = tuple(list(band[:, j]) + [str(i)])
            hashbuckets[band_id].add(j)
    candidate_pairs = set()
    for bucket in hashbuckets.values():
        if len(bucket) > 1:
            for pair in itertools.combinations(bucket, 2):
                candidate_pairs.add(pair)
    return candidate_pairs



# set p = 0.3 arbitrarily
p = 0.3
n = 120
b = 30
r = 4

# see how many candidate pairs we got right!
min_hasher = HashManager(shingle_dict,
                             n_signature=n)

sig_mat = min_hasher(doc_shingles)

true_score_dict = trueSimScores(doc_shingles)
approx_score_dict = sigSimScores(sig_mat)
print('True pairs:\t', candidatePairs(true_score_dict, p))
print("True LSH pairs:\t", bandedCandidatePairs(sig_mat, b, r))
print("Fast LSH pairs:\t", fastCandidatePairs(sig_mat, b, r))
print("MinHash pairs:\t", candidatePairs(approx_score_dict, p))

True pairs:	 {(3, 4), (2, 3)}
True LSH pairs:	 {(1, 3)}
Fast LSH pairs:	 {(1, 3)}
MinHash pairs:	 {(3, 4), (1, 3), (2, 3)}


# cosine similarity

In [18]:
def cossim(u,v):
    norm = np.linalg.norm(u)*np.linalg.norm(v)
    cosine = u@v/norm
    ang = np.arccos(cosine)
    return 1-ang/np.pi

## Generate data

We create N random vectors of dimension D. We then reassign 5 of these to be artifically close to another set of 5. This is done by setting them equal and adding a small amount of noise.

By artificially making some vectors closer to others, we can easily tune the target similarity threshold, allowing us to experiment better.

In [19]:
# get some random data points
N = 1000
D= 50
A = np.random.randn(N,D)
# artificially make some similar to others
A[5] = A[99] +np.random.randn(D)*0.05
A[20] = A[85] +np.random.randn(D)*0.15
A[13] = A[19] +np.random.randn(D)*0.25
A[56] = A[71] +np.random.randn(D)*0.5
A[45] = A[49] +np.random.randn(D)*0.66

In [20]:
true_pairs_dict = {}

thresh = 0.8

start = time.time()
for (i,u),(j,v) in itertools.combinations([(i,x) for i,x in enumerate(A)],2):
    val = cossim(u,v)
    if val > thresh:
        true_pairs_dict[(i,j)] = val
t_brute = time.time()-start

# save just the keys without the values. Easier to compare later to LSH
true_pairs = set(true_pairs_dict.keys())

print(f"Brute force calculation time: {t_brute:.3f}")
print(f"Discovered pairs:")
for k, v in true_pairs_dict.items():
    print(f"Pair: {k},\tSimilarity: {v:.2f}.")

Brute force calculation time: 6.688
Discovered pairs:
Pair: (5, 99),	Similarity: 0.98.
Pair: (13, 19),	Similarity: 0.91.
Pair: (20, 85),	Similarity: 0.96.
Pair: (45, 49),	Similarity: 0.82.
Pair: (56, 71),	Similarity: 0.84.


## LSH

### Find r, b

We must have that D%b==0, that is the number of bands divides the dimension. That's our only constraint. Remember, higher n is always better!

There's no magic here, we just trial and error some combinations until we get near the desired threshold, in this case 0.75 (chosen in the previous cell)

In [21]:
b, r = 50, 18

n = b*r
print(f"Transition probability: {(1/b)**(1/r):.2f}")

Transition probability: 0.80


### Compute LSH pairs

Let $S$ be the signature matrix and $R$ be the matrix of random hyperplanes.

In the code implementation, we work with the transpose of our equations. This means that $S = (N,D)$, and so we have to split into bands across axis=1

In [22]:
# Compute signature matrix
R = A@np.random.randn(D,n)
S = np.where(R>0,1,0)

print(R.shape, S.shape, R, S)

# Break into bands
S = np.split(S, b, axis=1)

print(type(S))

(1000, 900) (1000, 900) [[ 16.05679559  15.1420068   -1.26975867 ...   9.23963997  -6.830037
    4.49195993]
 [ -4.35275537  17.61545347  -4.32625181 ...   1.14243069  -0.25788092
    8.90249096]
 [ -5.61447502  -3.51569053  -3.67893566 ...   6.53054196   6.61756631
    3.61881017]
 ...
 [  2.44747745   0.19899818 -13.13572009 ...  -6.52809827  -5.42219705
    5.17513203]
 [  2.7509689    0.40571123   3.80342116 ...  -3.38090496  -1.46190723
   16.86744729]
 [ -8.55614126  10.14898437   4.77019353 ...   6.17988156   2.46681994
    7.68040769]] [[1 1 0 ... 1 0 1]
 [0 1 0 ... 1 0 1]
 [0 0 0 ... 1 1 1]
 ...
 [1 1 0 ... 0 0 1]
 [1 1 1 ... 0 0 1]
 [0 1 1 ... 1 1 1]]
<class 'list'>


In [23]:
start = time.time()

# Compute signature matrix
R = A@np.random.randn(D,n)
S = np.where(R>0,1,0)

# Break into bands
S = np.split(S, b, axis=1)

# column vector to convert binary vector to integer e.g. (1,0,1)->5
binary_column = 2**np.arange(r).reshape(-1,1)

# convert each band into a single integer, 
# i.e. convert band matrices to band columns
S = np.hstack([M@binary_column for M in S])

# Every value in the matrix represents a hash bucket assignment
# For every bucket in row i, add index i to that bucket
d = collections.defaultdict(set)
with np.nditer(S,flags=['multi_index']) as it:
    for x in it:
        d[int(x)].add(it.multi_index[0])
        
# For every bucket, find all pairs. These are the LSH pairs.
candidate_pairs = set()
for k,v in d.items():
    if len(v) > 1:
        for pair in itertools.combinations(v,2):
            candidate_pairs.add(tuple(sorted(pair)))

# Finally, perform the actually similarity computation
# to weed out false positive
lsh_pairs = set()
for (i,j) in candidate_pairs:
    if cossim(A[i],A[j]) > thresh:
        lsh_pairs.add((i,j))
        
t_lsh = time.time()-start

print(f"LSH calculation time: {t_lsh:.3f}")

LSH calculation time: 0.209


In [24]:
pairs_true = set(true_pairs_dict.keys())

In [25]:
print(f"t_brute: {t_brute:.3f}\t t_lsh: {t_lsh:.3f}. Speed-up: {t_brute/t_lsh:.0f}x")
print("True pairs: ", pairs_true)
print("LSH pairs: ", lsh_pairs)
print("Candidate pairs: " ,
      len(candidate_pairs),
      "False negatives: ", 
      len(pairs_true-lsh_pairs),
     sep='\n')

t_brute: 6.688	 t_lsh: 0.209. Speed-up: 32x
True pairs:  {(56, 71), (20, 85), (45, 49), (13, 19), (5, 99)}
LSH pairs:  {(20, 85), (45, 49), (13, 19), (5, 99)}
Candidate pairs: 
4915
False negatives: 
1


# Refactor Cosine Similarity 

In [27]:
from typing import Tuple, Set
from scipy.sparse import linalg as spl
from numpy import linalg as LA 

class RandomProjectionLSH():
    
    def __init__(self,
                n_signature : int,
                n_input_dimension : int,
                b : int,
                r : int,
                threshold = float,
                seed = int):
        """
        Consturct Random Projection LSH to find the candidate pair
        
        """
        assert n_signature == b * r, 'signatures should be bands (b) * rows per band (r)'
        np.random.seed(seed)
        
        self.n_input_dimension = n_input_dimension
        self.n_signature = n_signature
        self.threshold = threshold
        self.b = b
        self.r = r
        # create a gaussion distributed matrix
        self.hyperplanes = np.random.randn(n_input_dimension, n_signature)
        self.signature = None
        self.n_observation = None
    
    def fit(self, input_matrix : np.ndarray):
        """
        input_matrix (np.ndarray) - (n_observations, n_input_dimension)
        
        Hint:
        
        Use input_matrix.dot(self.hyperplane) instead of 
        np.dot(input_matrix, self.hyperplane) due to some implementation details
        https://stackoverflow.com/questions/33817189/why-is-vector-dot-product-slower-with-scipys-sparse-csr-matrix-than-numpys-den
        
        """
        self.n_observation = input_matrix.shape[0]
        self.input_matrix = input_matrix
        # build signature , shape (n_observation, n_signature)
        self.signature = input_matrix.dot(self.hyperplanes)
        self.signature = np.reshape(self.signature,
                                    (self.n_observation, self.b, self.r))
        self.signature = self.signature >= 0
        self.hashed_values = self._generate_hash()
        
    def find_candidates(self, dtype='dense') -> Tuple[Set, int]:
        """
        Apply OR logic
        
        In each band
        check is there other observation in the same bucket
        if yes, there is a cindidate
            output : 
            
            duplicates : candidate pair by id
            n_candidates : for all ids, number of candidates
        """
        if dtype == 'dense':
            cos_sim = self._consine_similarity_dense
        elif dtype == 'sparse':
            cos_sim = self._cosine_similarity_sparse
            
        candidates = []
        for i in range(self.b):
            un_values, counts = np.unique(self.hashed_values[:, i],
                                          return_counts=True) # get unique integers and their count
            
            non_unique_values = un_values[counts > 1] # identify integers which appear more than once
            
            for val in non_unique_values: # store duplicate integers as candidates
                candidates.append(np.where(
                    self.hashed_values[:, i] == val)[0])
        
        n_candidates = len(candidates)

        # get cosine distance for every candidate
        dist = [cos_sim(self.input_matrix, cand[0], cand[1])
                for cand 
                in candidates] # get the distance for all candidates

        # get duplicates
        duplicates = set(
            (candidates[i][0], candidates[i][1], dist[i])
            for i in range(len(dist)) 
            if dist[i] >= self.threshold) 

        return duplicates, n_candidates
    
    def _consine_similarity_dense(self, X, i, j) -> float:
        
        return np.dot(X[i], X[j]) / (LA.norm(X[i]) * LA.norm(X[j]))
    
    def _cosine_similarity_sparse(self, X, i, j) -> float:
        """
        This function is for sparse vector calculations
        """
        
        i_norm = spl.norm(X[i])
        j_norm = spl.norm(X[j])
        ij_dot = X[i].dot(X[j].T)[0, 0]

        return ij_dot/(i_norm*j_norm)
    
    def _generate_hash(self) -> np.ndarray:
        """
        Apply and logic for rows per band
        
        Hash all of element in 
        the signature matrix(n_observations, b, r)
        into bucket by element-wise multify between
        
        signature and bucket_id
        
        output : 
            signature_matrix (band) shape : (n_oberservation, b)
        """
        vals = 2**np.repeat(
            [np.repeat([np.arange(self.r)], self.b, axis=0)],
            self.n_observation,
            axis=0)
        
        # elementwise product
        hashed_values = np.multiply(self.signature, vals)
        hashed_values = np.sum(hashed_values, axis=2)
        return hashed_values
    
    

In [28]:
A.shape

(1000, 50)

In [29]:
rplsh = RandomProjectionLSH(
    n_signature=900,
    n_input_dimension=50,
    b=50,
    r=18,
    threshold = 0.75,
    seed=42
)

rplsh.fit(A)

similar, n_similar = rplsh.find_candidates()

In [30]:
print(similar, n_similar, sep='\n')

{(20, 85, 0.9924562162789341), (56, 71, 0.8818060776779674), (13, 19, 0.9602911075892671), (5, 99, 0.9982835435718197)}
350


In [31]:
# r, b, n = (18, 50, 1000)

# # this is signatures
# signature_r_b = np.repeat([np.arange(r)], b, axis=0)
# # building matrix M
# vals = 2 ** np.repeat([signature_r_b], n, axis=0)

In [32]:
# np.multiply(rplsh.signature, vals).shape

In [33]:
# rplsh.input_matrix.shape

In [34]:
# rplsh.hashed_values[:, 0].shape

In [35]:
# Pair: (5, 99),	Similarity: 0.99.
# Pair: (13, 19),	Similarity: 0.93.
# Pair: (20, 85),	Similarity: 0.97.
# Pair: (45, 49),	Similarity: 0.81.
# Pair: (56, 71),	Similarity: 0.84.

In [ ]:
# True pairs:  {(56, 71), (20, 85), (45, 49), (13, 19), (5, 99)}
# LSH pairs:  {(20, 85), (45, 49), (13, 19), (5, 99)}